# Jupyter Notebook to HTML Converter

This notebook allows you to convert a Jupyter Notebook (`.ipynb` file) to an HTML file in Google Colab. It handles potential issues with interactive widgets (e.g., `ipywidgets`) that may cause errors during conversion. Follow these steps:

1. Run the cell below.
2. Upload your `.ipynb` file when prompted.
3. The script will clean the notebook, convert it to HTML, and automatically download the HTML file.

**Note**: The installation of required libraries (`nbconvert`, `jupyter`, `nbformat`) happens silently in the background. If you encounter errors, check the output messages for details.
```

In [ ]:
#@title Run Conversion { display-mode: "form" }
# Install required libraries
!pip install nbconvert jupyter nbformat > /dev/null 2>&1

import nbformat
import os
from google.colab import files
from nbconvert import HTMLExporter
from nbconvert.preprocessors import ClearOutputPreprocessor, ExecutePreprocessor

# Function to clean widget metadata from notebook
def clean_notebook_widgets(input_file, cleaned_file):
    try:
        with open(input_file, 'r', encoding='utf-8') as f:
            nb = nbformat.read(f, as_version=4)

        # Remove widget state metadata
        for cell in nb.cells:
            if 'outputs' in cell:
                for output in cell.outputs:
                    if 'data' in output and 'application/vnd.jupyter.widget-view+json' in output.data:
                        # Remove widget-related outputs
                        output.data.pop('application/vnd.jupyter.widget-view+json', None)
                        output.metadata.pop('widgets', None)
            if 'metadata' in cell and 'widgets' in cell.metadata:
                del cell.metadata['widgets']

        # Clear notebook-level widget metadata if present
        if 'metadata' in nb and 'widgets' in nb.metadata:
            del nb.metadata['widgets']

        # Save cleaned notebook
        with open(cleaned_file, 'w', encoding='utf-8') as f:
            nbformat.write(nb, f)
        print(f"Cleaned notebook saved as: {cleaned_file}")
        return True
    except Exception as e:
        print(f"Error cleaning notebook: {str(e)}")
        return False

# Function to convert .ipynb to HTML
def convert_ipynb_to_html(input_file, output_file):
    try:
        # Check if input file exists
        if not os.path.exists(input_file):
            raise FileNotFoundError(f"The file {input_file} does not exist.")

        # Create a cleaned version of the notebook
        cleaned_file = f"cleaned_{input_file}"
        if not clean_notebook_widgets(input_file, cleaned_file):
            print("Proceeding with original notebook due to cleaning failure.")
            cleaned_file = input_file

        # Initialize HTML exporter
        html_exporter = HTMLExporter(template_name='lab')

        # Optionally clear outputs to avoid widget rendering issues
        preprocessor = ClearOutputPreprocessor()
        execute_preprocessor = ExecutePreprocessor(timeout=600, kernel_name='python3')

        # Read the cleaned notebook
        with open(cleaned_file, 'r', encoding='utf-8') as f:
            nb = nbformat.read(f, as_version=4)

        # Convert to HTML
        (body, resources) = html_exporter.from_notebook_node(nb)

        # Save HTML output
        html_file = output_file + '.html'
        with open(html_file, 'w', encoding='utf-8') as f:
            f.write(body)

        # Check if HTML file was created
        if os.path.exists(html_file):
            print(f"Conversion successful! HTML file created: {html_file}")
            files.download(html_file)
        else:
            print("Error: HTML file was not created.")

    except Exception as e:
        print(f"An error occurred during conversion: {str(e)}")
        # Fallback: Try nbconvert command-line as an alternative
        print("Attempting fallback conversion using nbconvert command...")
        try:
            !jupyter nbconvert --to html {cleaned_file} --output {output_file}
            html_file = output_file + '.html'
            if os.path.exists(html_file):
                print(f"Fallback conversion successful! HTML file created: {html_file}")
                files.download(html_file)
            else:
                print("Fallback conversion failed: HTML file was not created.")
        except Exception as fallback_e:
            print(f"Fallback conversion failed: {str(fallback_e)}")

# Upload your .ipynb file
print("Please upload your .ipynb file:")
uploaded = files.upload()

# Get the uploaded file name
input_file = list(uploaded.keys())[0]
output_file = os.path.splitext(input_file)[0]  # Remove .ipynb extension for output

# Convert the uploaded notebook to HTML
convert_ipynb_to_html(input_file, output_file)